# CNN GTSRB Dataset: Traffic Sign Detection


Run for All Imports

In [1]:
import os
import cv2
import numpy as np 
import sys
# import tensorflow as tf

import torch
import torch.nn as nn
import torch.nn.functional as F


import torch
import torchvision
import torchvision.transforms as transforms
from sklearn.model_selection import train_test_split

#Cuda
from numba import jit, cuda
import numpy as np
# to measure exec time
from timeit import default_timer as timer 

from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

import torch.optim as optim

import matplotlib.pyplot as plt
import numpy as np

from trafficsign import TrafficSignDataset


Loading the Traffic sign detection [GTSRB] Dataset

In [2]:
import os
import random

import numpy as np
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
from tqdm.auto import tqdm

# Ensure deterministic behavior
torch.backends.cudnn.deterministic = True
random.seed(hash("setting random seeds") % 2**32 - 1)
np.random.seed(hash("improves reproducibility") % 2**32 - 1)
torch.manual_seed(hash("by removing stochasticity") % 2**32 - 1)
torch.cuda.manual_seed_all(hash("so runs are repeatable") % 2**32 - 1)



/Users/rental/ecen493r/Project_3/.venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
EPOCHS = 2
IMG_WIDTH = 30
IMG_HEIGHT = 30
NUM_CATEGORIES = 43
TEST_SIZE = 0.1
BATCH_SIZE = 4

In [4]:
import wandb

wandb.login()



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: 3arjunsingh3 (stark-industries). Use `wandb login --relogin` to force relogin


True

In [5]:
config = dict(
    epochs=5,
    classes=10,
    kernels=[16, 32],
    batch_size=4,
    learning_rate=0.005,
    dataset="GTSRB",
    architecture="CNN")

In [6]:
sweep_config = {
    'method': 'grid'    
    }

In [7]:
metric = {
    'name': 'loss',
    'goal': 'minimize'   
    }

sweep_config['metric'] = metric

In [8]:
parameters_dict = {
    'optimizer': {
        'values': ['adam', 'sgd']
        },
    'epochs':{
        'values':[2,4,8,10,14]
    },
    'lr':{
        'values':[0.1,0.01,0.001,0.0001,0.00001]
    },
    'batch_size':{
        'values':[10,20,32,64,128]
    },
}


sweep_config['parameters'] = parameters_dict

In [9]:
def model_pipeline(config=None):

    # tell wandb to get started
    with wandb.init(project="pytorch-demo", config=config):
      # access all HPs through wandb.config, so logging matches execution!
      config = wandb.config

      # make the model, data, and optimization problem
      model, train_loader, test_loader, criterion, optimizer = make(config)
      print(model)

      # and use them to train the model
      train(model, train_loader, criterion, optimizer, config)

      # and test its final performance
      test(model, test_loader)

    return model

In [10]:
import torch.nn.functional as F

class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.dropout1 = nn.Dropout2d(0.5)  # Add dropout after the first convolutional layer
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.dropout2 = nn.Dropout2d(0.5)  # Add dropout after the second convolutional layer
        self.fc1 = nn.Linear(256, 120)
        self.dropout3 = nn.Dropout(0.5)  # Add dropout after the first fully connected layer
        self.fc2 = nn.Linear(120, 84)
        self.dropout4 = nn.Dropout(0.5)  # Add dropout after the second fully connected layer
        self.fc3 = nn.Linear(84, NUM_CATEGORIES)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.dropout1(x)  # Apply dropout after the first convolutional layer
        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout2(x)  # Apply dropout after the second convolutional layer
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout3(x)  # Apply dropout after the first fully connected layer
        x = F.relu(self.fc2(x))
        x = self.dropout4(x)  # Apply dropout after the second fully connected layer
        x = self.fc3(x)
        return x



# net = Net()
# net(images.reshape(1,3,30,30)).shape

In [11]:
def make(config):
    # Make the data
    
    images_directory = os.path.join(os.getcwd(),'gtsrb')


    gtsrb_transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
    ])
    
    
    train, validate, test = TrafficSignDataset(data_dir=images_directory, test_size=TEST_SIZE, random_seed=42,train=True , test=False,validate=False,transform=gtsrb_transform),TrafficSignDataset(data_dir=images_directory, test_size=TEST_SIZE, random_seed=42,train=False , test=False,validate=True,transform=gtsrb_transform), TrafficSignDataset(data_dir=images_directory, test_size=TEST_SIZE,random_seed=42,train=False , test=True,validate=False,transform=gtsrb_transform)
    train_loader = torch.utils.data.DataLoader(train, batch_size=config.batch_size,
                                          shuffle=True, num_workers=2)
    validate_loader = torch.utils.data.DataLoader(validate, batch_size=config.batch_size,
                                          shuffle=True, num_workers=2)
    test_loader = torch.utils.data.DataLoader(test, batch_size=config.batch_size,
                                          shuffle=True, num_workers=2)
    # print('Problem? No')
    # Make the model
    model = Net()

    # Make the loss and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = build_optimizer(model, config.optimizer, config.lr)
    
    #torch.optim.Adam(
     #   model.parameters(), lr=config.learning_rate)
    # print('Problem? No')
    return model, train_loader, validate_loader, criterion, optimizer

Defining Training Logic:


In [12]:
epochs = [2,3,4,5]
lr = [0.1,0.01,0.001,0.0001]
loss_functions = [
    nn.CrossEntropyLoss(),
    nn.MSELoss(),
    nn.NLLLoss(),
    nn.HingeEmbeddingLoss(),
    # Add other loss functions as needed
]

batch_sizes = [2,3,4,5]

In [13]:
def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer

In [14]:
def train(model, loader, criterion, optimizer, config):
    # Tell wandb to watch what the model gets up to: gradients, weights, and more!
    wandb.watch(model, criterion, log="all", log_freq=10)

    # Run training and track with wandb
    total_batches = len(loader) * config.epochs
    example_ct = 0  # number of examples seen
    batch_ct = 0
    
    for epoch in tqdm(range(config.epochs)):
        for _, data in enumerate(loader,0):
            
            images, labels = data
            # optimizer = build_optimizer(model, optimizerr, config.learning_rate)
            train_loss = train_batch(data, model, optimizer, criterion)
            example_ct +=  len(images)
            batch_ct += 1
        
            # Report metrics every 25th batch
            if _ % 2000 == 1999:
                train_log(train_loss, example_ct, epoch)
            
# Check if this works out or not, I may neet to change the .to(device) thing                
def train_batch(data, model, optimizer, criterion):
    images, labels = data
    
    # Forward pass ➡
    outputs = model(images)
    loss = criterion(outputs, labels)
    
    # Backward pass ⬅
    optimizer.zero_grad()
    loss.backward()

    # Step with optimizer
    optimizer.step()

    return loss

In [15]:
def train_log(loss, example_ct, epoch):
    # Where the magic happens
    wandb.log({"epoch": epoch, "train_loss": loss}, step=epoch)
    print(f"Loss after {str(example_ct).zfill(5)} examples: {loss:.3f}")

In [16]:
def test(model, test_loader):
    model.eval()

    # Run the model on some test examples
    with torch.no_grad():
        correct, total = 0, 0
        for data in test_loader:
            images, labels = data
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        print(f"Accuracy of the model on the {total} " +
              f"test images: {correct / total:%}")
        
        wandb.log({"loss": correct / total})

    # Save the model in the exchangeable ONNX format
    torch.onnx.export(model, images, "model.onnx")
    wandb.save("model.onnx")

In [17]:
sweep_id = wandb.sweep(sweep_config, project="Project-3-testing-300-nn")

Create sweep with ID: 7da485vm
Sweep URL: https://wandb.ai/stark-industries/Project-3-testing-300-nn/sweeps/7da485vm


In [18]:
# Build, train and analyze the model with the pipeline
# model = model_pipeline(sweep_config)
wandb.agent(sweep_id, model_pipeline, count=300)

wandb: Agent Starting Run: 304lt7co with config:
wandb: 	batch_size: 10
wandb: 	epochs: 2
wandb: 	lr: 0.1
wandb: 	optimizer: adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: WARNING Ignored wandb.init() arg project when running a sweep.


Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (dropout1): Dropout2d(p=0.5, inplace=False)
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (dropout2): Dropout2d(p=0.5, inplace=False)
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (dropout3): Dropout(p=0.5, inplace=False)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (dropout4): Dropout(p=0.5, inplace=False)
  (fc3): Linear(in_features=84, out_features=43, bias=True)
)


  0%|          | 0/2 [00:00<?, ?it/s]

Loss after 20000 examples: 3.676


 50%|█████     | 1/2 [00:53<00:53, 53.94s/it]

...

 20%|██        | 2/10 [01:27<05:47, 43.43s/it]

Loss after 63156 examples: 3.588


 30%|███       | 3/10 [02:06<04:51, 41.59s/it]

Loss after 84734 examples: 3.581


wandb: Ctrl + C detected. Stopping sweep.
